In [29]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import pandas as pd
from scipy import stats
from pydataset import data
import numpy as np
import env
import matplotlib.pyplot as plt
import os
import prepare
import acquire
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [95]:
titan_df = acquire.get_titanic_data()
titan_df = titan_df.drop(titan_df.columns[0], axis = 1)


In [96]:
titan_df = prepare.clean_titanic(titan_df)
titan_df['baseline'] = 0
titan_df.head()


,survived,pclass,sex,sibsp,parch,fare,embark_town,alone,sex_male,embark_town_Queenstown,embark_town_Southampton,baseline
0,0,3,male,1,0,7.2500,Southampton,0,1,0,1,0
1,1,1,female,1,0,71.2833,Cherbourg,0,0,0,0,0
2,1,3,female,0,0,7.9250,Southampton,1,0,0,1,0
3,1,1,female,1,0,53.1000,Southampton,0,0,0,1,0
4,0,3,male,0,0,8.0500,Southampton,1,1,0,1,0


In [48]:
titan_df = titan_df.drop(columns = ['sex','embark_town',])

In [49]:
titan_df_mod = titan_df.drop(columns = ['alone','embark_town_Southampton', 
                                   'embark_town_Queenstown','sex_male','parch', 'sibsp'])



In [50]:
titan_df_mod.head()

,survived,pclass,fare,baseline
0,0,3,7.2500,0
1,1,1,71.2833,0
2,1,3,7.9250,0
3,1,1,53.1000,0
4,0,3,8.0500,0


In [51]:
train, val, test = prepare.train_val_test(titan_df_mod,'survived')

In [52]:
(train.survived == train.baseline).mean()

0.6163723916532905

In [53]:
x_train = train.drop(columns = ['survived','baseline'])
y_train = train['survived']
x_val = val.drop(columns = ['survived','baseline'])
y_val = val['survived']
x_test = test.drop(columns = ['survived','baseline'])
y_test = test['survived']

Create a model that includes only age, fare, and pclass. Does this model perform better than your baseline?

In [81]:
seed = 42
logit2 = LogisticRegression(penalty = 'l1', random_state = seed, solver = 'liblinear', max_iter = 100)
logit = LogisticRegression(random_state = seed)

In [55]:
logit.fit(x_train, y_train)

LogisticRegression(random_state=42)

In [56]:
logit.score(x_train, y_train)

0.680577849117175

Include sex in your model as well. Note that you'll need to encode or create a dummy variable of this feature before including it in a model.

In [57]:
titan_df_mod2 = titan_df.drop(columns = ['alone','embark_town_Southampton', 
                                   'embark_town_Queenstown','parch', 'sibsp'])
titan_df_mod2.head()

,survived,pclass,fare,sex_male,baseline
0,0,3,7.2500,1,0
1,1,1,71.2833,0,0
2,1,3,7.9250,0,0
3,1,1,53.1000,0,0
4,0,3,8.0500,1,0


In [58]:
train, val, test = prepare.train_val_test(titan_df_mod2,'survived')

In [59]:
x_train = train.drop(columns = ['survived','baseline'])
y_train = train['survived']
x_val = val.drop(columns = ['survived','baseline'])
y_val = val['survived']
x_test = test.drop(columns = ['survived','baseline'])
y_test = test['survived']

In [60]:
logit.fit(x_train, y_train)

LogisticRegression(random_state=42)

In [61]:
logit.score(x_train, y_train)

0.7881219903691814

In [82]:
logit2.fit(x_train, y_train)

LogisticRegression(penalty='l1', random_state=42, solver='liblinear')

In [83]:
logit2.score(x_train, y_train)

0.6356340288924559

In [62]:
logit.score(x_val,y_val)

0.7985074626865671

Try out other combinations of features and models.

In [65]:
titan_df_mod3 = titan_df.drop(columns = ['alone','pclass', 
                                   'sex_male','parch', 'sibsp','fare'])
titan_df_mod3.head()

,survived,embark_town_Queenstown,embark_town_Southampton,baseline
0,0,0,1,0
1,1,0,0,0
2,1,0,1,0
3,1,0,1,0
4,0,0,1,0


In [66]:
train, val, test = prepare.train_val_test(titan_df_mod3,'survived')

In [67]:
x_train = train.drop(columns = ['survived','baseline'])
y_train = train['survived']
x_val = val.drop(columns = ['survived','baseline'])
y_val = val['survived']
x_test = test.drop(columns = ['survived','baseline'])
y_test = test['survived']

In [74]:
logit2.fit(x_train, y_train)

LogisticRegression(max_iter=200, penalty='l1', random_state=42,
                   solver='liblinear')

In [75]:
logit2.score(x_train, y_train)

0.6356340288924559

In [76]:
logit2.score(x_val, y_val)

0.6194029850746269

mod 4

In [84]:
def clean_titanic(df):
    '''
    this function cleans up the titanic db
    '''
    
    to_drop = ['class', 'embarked','deck','passenger_id', 'age']
    df.drop(columns = to_drop, inplace = True)
    df['embark_town'].fillna('Southampton',inplace = True)
    dummies = pd.get_dummies(df[['sex','embark_town']],drop_first = [True, True])
    df = pd.concat([df,dummies], axis = 1)
    return df


In [86]:
x_train = train['sibsp'],['alone'],['sex_male']
y_train = train['survived']
x_val = val['sibsp'],['parch'],['sex_male']
y_val = val['survived']
x_test = test['sibsp'],['parch'],['sex_male']
y_test = test['survived']